In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
groq_api_key = os.environ['GROQ_API_KEY']


In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults()

In [4]:
search_tool.invoke('who won the last US presidential elections?')

[{'url': 'https://www.270towin.com/2020_Election/results',
  'content': 'The 2020 United States presidential election was the 59th quadrennial presidential election, held on Tuesday, November 3, 2020. The Democratic ticket of former vice president Joe Biden and incumbent U.S. senator from California Kamala Harris defeated the Republican ticket of incumbent president Donald Trump and vice president Mike Pence.'},
 {'url': 'https://www.cnn.com/election/2020/results/president',
  'content': "Presidential Election Results and Electoral Map 2020 CNN Newsletters Work for CNN Follow CNN Politics Results Joe Biden wins election to be the 46th US President Pennsylvania’s 20 electoral votes put native son Joe Biden above the 270 needed to become the 46th President of the United States. Pennsylvania’s 20 electoral votes put native son Joe Biden above the 270 needed to become the 46th President of the United States. In Maine, two of four electoral votes go to the statewide winner and one electoral

## Prepare the RAG tool

* RAG tool loading data from a web page

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader('https://docs.smith.langchain.com/overview')
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vectorstore = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

* confirm that RAG works

In [9]:
retriever.invoke('how to upload a dataset')

[Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Run the evaluationexperiment_results = client.evaluate(    dummy_app, # Your AI system goes here    data=dataset, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={"version": "1.0.0", "revision_id": "beta"}, # Metadata about the experiment    max_concurrency=4,  # Add concurrency.)# Analyze the results via the UI or programmatically# If you have \'pandas\' installed you can view the results as a# pandas DataFrame by uncommenting below:# experiment_results.to_pandas()import { Client } from "langsmith";import { EvaluationResult, evaluate } from "langsmith/evaluation";const client = new Client();// Defin

In [10]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

## Now let´s add a custom tool

In [12]:
from langchain.agents import tool

@tool
def get_word_length_tool(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

get_word_length_tool.invoke("julio")

5

## Now we create the list of tools availabe

In [13]:
tools = [search_tool, retriever_tool, get_word_length_tool]

## Create the agent
* We will create an **OpenAI Functions Agent** (see the different Agent Types in the above section).

In [14]:
# Choose LLM model
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)


#### Define the Agent Prompt
* The Agent Prompt will guide the agent.
* For this example, we will use a pre-defined prompt from the LangSmith Prompt Hub. See the prompt definition [here.](https://smith.langchain.com/hub/hwchase17/openai-functions-agent)

In [15]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

#### Initialize the agent
* With the LLM, the prompt and the tools.

In [16]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm=model, tools=tools, prompt=prompt)

#### Initialize the AgentExecutor
* We combine the agent with the tools inside the AgentExecutor, which will repeatedly call the agent and execute tools.

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## We can now start using the agent
* Keep in mind that this is a very basic agent that will not remember previous interactions.

In [18]:
agent_executor.invoke({"input": "hi! who are you?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Hello! I'm an AI assistant here to help you with any questions or tasks you have. How can I assist you today?

> Finished chain.


{'input': 'hi! who are you?',
 'output': "Hello! I'm an AI assistant here to help you with any questions or tasks you have. How can I assist you today?"}

In [19]:
agent_executor.invoke({"input": "in less than 20 words, what is langsmith?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `langsmith_search` with `{'query': 'definition of LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith






Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using eithe

{'input': 'in less than 20 words, what is langsmith?',
 'output': 'LangSmith is a platform for building production-grade LLM applications with monitoring and evaluation capabilities.'}

In [22]:
agent_executor.invoke({'input':'who won last 2024 US presidential elections?'})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `tavily_search_results_json` with `{'query': '2024 US presidential election winner'}`


[{'url': 'https://www.bbc.co.uk/news/election/2024/us/results', 'content': 'US Presidential Election Results 2024 - BBC News Close menu BBC News Kamala Harris of the Democrat party has 27 electoral college votes. Donald Trump of the Republican party has 90 electoral college votes. Kamala Harris of the Democrat party has 10,652,749 votes (44.9%) Donald Trump of the Republican party has 12,812,102 votes (54.0%) US presidential election results 2024 US election 2024 Voting in some states is particularly hard to predict, with polls showing they could be won by the Republicans or the Democratic party. Voters in 11 states will also elect a governor. How to follow the US election on the BBC Path to 270: The states Harris and Trump need to win US election 2024 About the BBC'}, {'url': 'https://www.bbc.com/news/election/2024/us/results', 'content': 'US Presidential Election Results 2024 - BBC News

{'input': 'who won last 2024 US presidential elections?',
 'output': 'Donald Trump won the 2024 US presidential election.'}

In [23]:
agent_executor.invoke({"input": "How many letters are in the word AIAccelera?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `get_word_length_tool` with `{'word': 'AIAccelera'}`


10The word "AIAccelera" contains 10 letters.

> Finished chain.


{'input': 'How many letters are in the word AIAccelera?',
 'output': 'The word "AIAccelera" contains 10 letters.'}

## Adding memory to the agent
* Right now, our basic agent does not remember previous interactions. Technically, we say that our agent is "stateless".
* If we want the agent to remember previous interactions, we could achieve it message by message, or automatically. For the sake of simplicity, let's first see how to do it message by message:

In [25]:
# Here we pass in an empty list of messages 
# for chat_history because it is the first 
# message in the chat
agent_executor.invoke(
    {
        "input": "hi! my name is julio", 
        "chat_history": []
    })

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Hello Julio! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is julio',
 'chat_history': [],
 'output': 'Hello Julio! How can I assist you today?'}

In [26]:
from langchain_core.messages import AIMessage, HumanMessage

In [27]:
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is julio"),
            AIMessage(content="Hello Julio! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Your name is Julio. How can I assist you further, Julio?

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is julio', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Julio! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'input': "what's my name?",
 'output': 'Your name is Julio. How can I assist you further, Julio?'}

* The previous way is simple but not very practical for real agents. Let's now see how we can keep track of previous messages automatically:

In [28]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

In [30]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

agent_with_chat_history.invoke(
    {"input": "hi! I'm julio"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Hello Julio! How can I assist you today?

> Finished chain.


{'input': "hi! I'm julio",
 'chat_history': [],
 'output': 'Hello Julio! How can I assist you today?'}

In [31]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Your name is Julio! How can I help you, Julio?

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm julio", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Julio! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Julio! How can I help you, Julio?'}

## Basic Agent Operations: How to Guides
* [How to build an agent with a custom tool and memory.](https://python.langchain.com/docs/modules/agents/how_to/custom_agent/)
* [How to add streaming.](https://python.langchain.com/docs/modules/agents/how_to/streaming/)
* [How to return structured output.](https://python.langchain.com/docs/modules/agents/how_to/agent_structured/)
* [How to run an agent as an iterator.](https://python.langchain.com/docs/modules/agents/how_to/agent_iter/)
* [How to handle parsing errors.](https://python.langchain.com/docs/modules/agents/how_to/handle_parsing_errors/)
* [How to access intermediate steps.](https://python.langchain.com/docs/modules/agents/how_to/intermediate_steps/)
* [How to cap the max number of iterations.](https://python.langchain.com/docs/modules/agents/how_to/max_iterations/)
* [How to add timeouts for agents.](https://python.langchain.com/docs/modules/agents/how_to/max_time_limit/)